In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd


In [2]:
df = pd.read_csv(r"C:\Users\domin\Desktop\Year 2 Block C\2023-24c-fai2-adsai-DominikSzewczyk224180\Datasets\emotion_data_merged.csv")
df = df[df['emotion'] != 'neutral']
df= df[:100]

df = pd.read_csv(r"C:\Users\domin\Desktop\Year 2 Block C\2023-24c-fai2-adsai-DominikSzewczyk224180\Datasets\generated data\Generated_data_multiclass_1.csv")


In [4]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['sentence'], df['emotion'], test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)

label_map = {"happiness": 0, "sadness": 1, "anger": 2, "surprise": 3, "fear": 4, "disgust": 5}
train_labels = train_labels.map(label_map)
val_labels = val_labels.map(label_map)


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
)).shuffle(len(train_texts)).batch(16)

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
)).batch(16)

In [6]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
model.fit(train_dataset, epochs=3, validation_data=val_dataset)

Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



50/50 [==============================] - 361s 6s/step - loss: 1.5835 - accuracy: 0.3990 - val_loss: 0.9483 - val_accuracy: 0.8071
Epoch 2/3
50/50 [==============================] - 235s 5s/step - loss: 0.4491 - accuracy: 0.9009 - val_loss: 0.2180 - val_accuracy: 0.9340
Epoch 3/3
50/50 [==============================] - 233s 5s/step - loss: 0.1067 - accuracy: 0.9771 - val_loss: 0.1913 - val_accuracy: 0.9391


In [8]:
model.save_pretrained('./model5/')

In [9]:

val_pred_logits = model.predict(val_dataset)
val_pred_labels = tf.argmax(val_pred_logits.logits, axis=1)

val_accuracy = accuracy_score(val_labels, val_pred_labels)
val_f1 = f1_score(val_labels, val_pred_labels, average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1)

13/13 [==============================] - 32s 791ms/step
Validation Accuracy: 0.9390862944162437
Validation F1 Score: 0.9395160494593161


In [10]:
test_df = pd.read_csv(r'C:\Users\domin\Desktop\Year 2 Block C\2023-24c-fai2-adsai-DominikSzewczyk224180\Datasets\test_multiclass.csv', sep = "\t")

test_encodings = tokenizer(test_df['sentence'].tolist(), truncation=True, padding=True)

test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings)))

test_pred_logits = model.predict(test_dataset.batch(16))
test_pred_labels = tf.argmax(test_pred_logits.logits, axis=1)

label_map_inverse = {v: k for k, v in label_map.items()}
predicted_emotions = [label_map_inverse[label] for label in test_pred_labels.numpy()]


output_df = pd.DataFrame({'id': test_df['id'], 'emotion': predicted_emotions})
output_df.to_csv('predictions_test_new_model_multiclass6.csv', index=False)  


90/90 [==============================] - 145s 1s/step
